In [5]:
# Necessary imports of functions and dictionaries
from dictionaries import *
from sample_sheet_functions import *
import pandas as pd
from io import StringIO


# Make controls so that are always at the low bottom 

# User downloads sample sheet
*Requires User* 
- to pick for which tables 
- how many if 96-wells
- Project name, experiment name, additional comments

_To add_:
- Make function to add 48 well plate at bottom of sample sheet

- User can choose csv or xlsx (xlsx is going to be coloured)
- User can choose to add a 48 well + 98 well • n
- Handle special orders
    + Eg for a 384 well - 3 plates have 6 controls and one has none
- User has possibility to add a file with sample name and additional comments and that gets mapped to the final sample sheet

In [6]:
# Essential input
plate_wells_number_options = ['24', '48', '96', '96+48', '96+24'] ## Need to check that no indexes clash for 48 well plates and 96f 
how_many_96_plates_options = [1, 'up to', 8]
project_name = ''

# Non essential
experiment_name = ''
additional_comments = ''
file_format = ['csv', 'xlsx'] #def csv

In [7]:
# Simulating user input up to this point
user_plate = '96'
user_plate_number = 5
user_project_name = 'my_project'
user_experiment_name = 'my_experiment'
user_additional_comments = 'my_comments'
user_file_format = 'csv'
order_number = '' #!!!!!!!!!!!!!!!!!!!! add somewhere!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [8]:
# Add option for the CSV format
def wrapper_make_sample_sheet(plate_type, how_many, project_name, file_format='csv'):
    sample_sheet = None
    filename = ''
    # If the plate type is 96, call the function for 96, if it's not check, which one and give the right boolean to the 24/48 function
    if plate_type == '96':
        filename=f'96_well_template_{project_name}.{file_format}'
        sample_sheet = make_96_well_sample_sheet(filename, how_many)
    elif plate_type == '24':
        filename=f'24_well_template_{project_name}.{file_format}'
        sample_sheet = make_24_48_well_sample_sheet(filename, True)
    elif plate_type == '48':
        filename=f'48_well_template_{project_name}.{file_format}'
        sample_sheet = make_24_48_well_sample_sheet(filename, False)
    else:
        print('I DO NOT RECOGNISE THIS PLATE TYPE')
        raise Exception('Wrong plate format required!')
    
    if file_format == 'xlsx':
        # Wrapper to write file in xlsx
        pass
        # sample_sheet = wrapper(sample_sheet)


    #Check that file does not exist already
    # if os.path.isfile(filename):
    #     raise Exception('This file already exists') 

    # Save in file the string for the sample sheet (this will probably need to be modified for Dash, not sure how it handles the download of files yet)
    with open(filename, 'w') as file:
        file.write(sample_sheet)
    


In [9]:
# Execute make sample sheet with user specified variables
wrapper_make_sample_sheet(user_plate, user_plate_number, user_project_name)

# User uploads sample sheet
#### Displays the layout of the uploaded sample sheet(s) to the user
*Persisting from before/can be updated*
- Experiment_name
- Additional_comments

*Requires user*
- Pick plate flavours

_In background_:
- Checks number of samples
- Checks number of blanks/controls
- Checks no samples start with underscore (_)

_To add_:
- User can upload CSV or XLSX

_Processes:_
- Read file in, extract different tables and display to users
- Associate each table to a plate flavour
- Associate dictionaries and produce generic melted sample sheet

In [6]:
# User reuploaded file, in this case the file name is:
user_file_name_on_reupload = '96_well_template_4_plates.csv'

# Persisting from before
user_experiment_name = user_experiment_name
user_additional_comments = user_additional_comments

In [7]:
# The file gets read, and the table(s) get loaded on pandas. The tables are then displayed to the user, where they're prompted to select the plate flavours

#Here add read extension and different read in methods
sheet = pd.read_csv(user_file_name_on_reupload, index_col=0)

tables = get_tables(sheet)

reports = [check_table(x) for x in tables]

# print(reports)
for x in range(len(reports)):
    if reports[x]['something_wrong']:
        print(f'OOOOPPPPSSS something went wrong here in table {x+1}, this is the report: {reports[x]}')


# DO CHECKS HERE!!!!!!!!!!!!!!!!!!!!!!!!!!

print(f'I need you to select exactly {len(tables)} plate flavours')

I need you to select exactly 4 plate flavours


In [8]:
# Here the user inputs what flavours he wants for the tables, in this case:
user_flavours_selection = ['G', 'H', 'I', 'F'] 

# In the next section the user has to select the machinery so:
possible_machinery_selection = ['hi_seq', 'mini_seq', 'mi_seq', 'next_seq']

user_machinery_selection = 'next_seq' #(from radio button on dash)

# Here there are two solutions, the first was the first one developed, which works fine, the second one is the one that produces the massive sample sheet at the end!

## Old solution

In [9]:
# Function to combine header and melted table
def combine_into_sample_sheet(tables, selected_flavours, machinery_selection, user_project_name, user_experiment_name = '', user_additional_comments=''):
    header = ''
    dfs = []
    filename = ''
    if machinery_selection == 'hi_seq':
        header = make_hiseq_header()
        dfs = loop_and_stack(tables, selected_flavours)
        filename = f'{user_project_name}_SR_HS.csv'
    
    elif machinery_selection == 'mini_seq':
        header = make_miniseq_header(user_experiment_name)
        dfs = loop_and_stack(tables, selected_flavours, mini_seq=True)
        filename = f'{user_project_name}_MiniSeq.csv'

    elif machinery_selection == 'mi_seq':
        header = make_miseq_header(user_project_name, user_experiment_name, user_additional_comments)
        dfs = loop_and_stack(tables, selected_flavours)
        filename = f'{user_project_name}_SR_MS.csv'

    elif machinery_selection == 'next_seq':
        header = make_nextseq_header(user_project_name, user_experiment_name, user_additional_comments)
        dfs = loop_and_stack(tables, selected_flavours, next_seq=True)
        filename = f'{user_project_name}_PE_NS.csv'    
    
    else:
        print('Something went wrong, match not made')
        raise Exception('This machinery is not supported yet!')


    # Need to concatenate, remove headers, re-index and remove standard index from file
    merged_dfs = concatenate_and_reindex(dfs)     
    
    # Write header and dfs to csv file
    with open(filename, 'w') as file:
        file.write(header)

        merged_dfs.to_csv(file, index=False, header=False)

    print('DONE') # Or return some kind of feedback to dash





combine_into_sample_sheet(tables, user_flavours_selection, user_machinery_selection, user_project_name, user_experiment_name, user_additional_comments)




DONE


# New Solution, outputs classic sample sheet and complete one with all primers combinations



# BRUH YOU FORGOT TO HANDLE THE MACHINERY INSTRUMENT!!!! FIX THAT

In [10]:
tables #from above

[    R01 R02 R03 R04 R05 R06 R07 R08 R09  R10  R11  R12
 F01  A1  A2  A3  A4  A5  A6  A7  A8  A9  A10  A11  A12
 F02  B1  B2  B3  B4  B5  B6  B7  B8  B9  B10  B11  B12
 F03  C1  C2  C3  C4  C5  C6  C7  C8  C9  C10  C11  C12
 F04  D1  D2  D3  D4  D5  D6  D7  D8  D9  D10  D11  D12
 F05  E1  E2  E3  E4  E5  E6  E7  E8  E9  E10  E11  E12
 F06  F1  F2  F3  F4  F5  F6  F7  F8  F9  F10  F11  F12
 F07  G1  G2  G3  G4  G5  G6  G7  G8  G9  G10  G11  G12
 F08  H1  H2  H3  H4  H5  H6  H7  H8  H9  H10  H11  H12,
     R01 R02 R03 R04 R05 R06 R07 R08 R09  R10  R11  R12
 F01  I1  I2  I3  I4  I5  I6  I7  I8  I9  I10  I11  I12
 F02  J1  J2  J3  J4  J5  J6  J7  J8  J9  J10  J11  J12
 F03  K1  K2  K3  K4  K5  K6  K7  K8  K9  K10  K11  K12
 F04  L1  L2  L3  L4  L5  L6  L7  L8  L9  L10  L11  L12
 F05  M1  M2  M3  M4  M5  M6  M7  M8  M9  M10  M11  M12
 F06  N1  N2  N3  N4  N5  N6  N7  N8  N9  N10  N11  N12
 F07  O1  O2  O3  O4  O5  O6  O7  O8  O9  O10  O11  O12
 F08  P1  P2  P3  P4  P5  P6  P7  P8  P9  P10  

In [11]:
# Read in the complete manifest/sample sheet file
complete_sample_sheet = pd.read_csv(StringIO(complete_sample_sheet_string), index_col=0)
complete_sample_sheet

,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description
0,1,__Plate_E_A01,E,A01,R902,TCCTCAGCC,F904,CTAATGCAC,NaN,NaN
1,2,__Plate_E_B01,E,B01,R902,TCCTCAGCC,F906,GATGGTTGT,NaN,NaN
2,3,__Plate_E_C01,E,C01,R902,TCCTCAGCC,F908,AAGGTGTTT,NaN,NaN
3,4,__Plate_E_D01,E,D01,R902,TCCTCAGCC,F916,AATTCGGAA,NaN,NaN
4,5,__Plate_E_E01,E,E01,R902,TCCTCAGCC,F927,GACCTTGCT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
763,764,__Plate_L_D12,L,D12,R996,GATAGCGGA,F951,ATACAGTTC,NaN,NaN
764,765,__Plate_L_E12,L,E12,R996,GATAGCGGA,F953,TTGGATTCA,NaN,NaN
765,766,__Plate_L_F12,L,F12,R996,GATAGCGGA,F954,CCGATGTTC,NaN,NaN
766,767,__Plate_L_G12,L,G12,R996,GATAGCGGA,F962,TCTGTCATT,NaN,NaN


In [12]:
# So pretty much you need to
# - map sample names based on well position
# - substitute sample names in massive sample sheet
# - if next_seq remove index2 col and change index2_reverse to index2
#       - Else remove index2_reverse
# - If miniSeq has to give fewer ones

In [13]:
# Map sample names based on well position
mapped_sheets = [map_sample_to_well(x, y) for x, y in zip(tables, user_flavours_selection)]
mapped_sheets

[   Sample_Name Sample_Well Sample_Plate
 0           A1         A01            G
 1           A2         A02            G
 2           A3         A03            G
 3           A4         A04            G
 4           A5         A05            G
 ..         ...         ...          ...
 91          H8         H08            G
 92          H9         H09            G
 93         H10         H10            G
 94         H11         H11            G
 95         H12         H12            G
 
 [96 rows x 3 columns],
    Sample_Name Sample_Well Sample_Plate
 0           I1         A01            H
 1           I2         A02            H
 2           I3         A03            H
 3           I4         A04            H
 4           I5         A05            H
 ..         ...         ...          ...
 91          P8         H08            H
 92          P9         H09            H
 93         P10         H10            H
 94         P11         H11            H
 95         P12         H12    

In [14]:
# Concatenate Sample Sheets
user_sample_sheet = stack_data_frames(mapped_sheets)
user_sample_sheet


,Sample_Name,Sample_Well,Sample_Plate
0,A1,A01,G
1,A2,A02,G
2,A3,A03,G
3,A4,A04,G
4,A5,A05,G
...,...,...,...
379,AF8,H08,F
380,AF9,H09,F
381,AF10,H10,F
382,AF11,H11,F


In [15]:
# Now iterate through the large manifest/sample sheet data frame and substitute sample names with user's sample names if well and plate position matches
complete = migrate_sample_name(complete_sample_sheet, user_sample_sheet)
complete # This is the large file with all the indexes combination

,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description
0,1,__Plate_E_A01,E,A01,R902,TCCTCAGCC,F904,CTAATGCAC,NaN,NaN
1,2,__Plate_E_B01,E,B01,R902,TCCTCAGCC,F906,GATGGTTGT,NaN,NaN
2,3,__Plate_E_C01,E,C01,R902,TCCTCAGCC,F908,AAGGTGTTT,NaN,NaN
3,4,__Plate_E_D01,E,D01,R902,TCCTCAGCC,F916,AATTCGGAA,NaN,NaN
4,5,__Plate_E_E01,E,E01,R902,TCCTCAGCC,F927,GACCTTGCT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
763,764,__Plate_L_D12,L,D12,R996,GATAGCGGA,F951,ATACAGTTC,NaN,NaN
764,765,__Plate_L_E12,L,E12,R996,GATAGCGGA,F953,TTGGATTCA,NaN,NaN
765,766,__Plate_L_F12,L,F12,R996,GATAGCGGA,F954,CCGATGTTC,NaN,NaN
766,767,__Plate_L_G12,L,G12,R996,GATAGCGGA,F962,TCTGTCATT,NaN,NaN


In [16]:
# Remove non-user sample names
complete_filtered = filter_df(complete)
complete_filtered # Large file with only user-specific rows


,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description
0,1,Y1,F,A01,R936,TGGAACGCT,F904,CTAATGCAC,NaN,NaN
1,2,Z1,F,B01,R936,TGGAACGCT,F906,GATGGTTGT,NaN,NaN
2,3,AA1,F,C01,R936,TGGAACGCT,F908,AAGGTGTTT,NaN,NaN
3,4,AB1,F,D01,R936,TGGAACGCT,F916,AATTCGGAA,NaN,NaN
4,5,AC1,F,E01,R936,TGGAACGCT,F927,GACCTTGCT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
379,380,T12,I,D12,R935,TAGCTCTTC,F915,GGTATCCTT,NaN,NaN
380,381,U12,I,E12,R935,TAGCTCTTC,F923,GGTTAAGGC,NaN,NaN
381,382,V12,I,F12,R935,TAGCTCTTC,F928,AGATTGAGT,NaN,NaN
382,383,W12,I,G12,R935,TAGCTCTTC,F930,GCGATAGCA,NaN,NaN


# Prompt the user for download
*Requires user*
- Pick which machine they need

_Processes_:
- Append header to melted table

In [17]:
# Simulating user input variables (which can be modified up to this point)
user_project_name = 'my_project'
user_experiment_name = 'my_experiment'
user_additional_comments = 'my_comments'
user_machinery_selection = 'next_seq'

In [18]:
finalise_sample_sheet(complete, user_machinery_selection, user_project_name, user_experiment_name, user_additional_comments, complete=True)

finalise_sample_sheet(complete_filtered, user_machinery_selection, user_project_name, user_experiment_name, user_additional_comments, complete=False)

DONE
DONE
